In [2]:
import numpy as np
import scipy as sc
import os
import csv
from scipy.spatial import distance
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, GridSearchCV
from copy import deepcopy
from helper_py import load_karypis
from sklearn.datasets import fetch_20newsgroups, load_iris
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from scipy.sparse import csr_matrix, vstack, hstack
import matplotlib.pyplot as plt
from sklearn.feature_selection import chi2, SelectKBest
#from tqdm import tnrange, tqdm_notebook
from sklearn.datasets import load_files
#import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC
from scipy.stats import ttest_ind

# Reuters

In [ ]:
def build_data_frame(rootdir):
    i=0
    data_frame = []
    data_frame_target = []
    target_hash, topic = {}, 0
    for root, subFolders, files in os.walk(rootdir):
        if(i==0):
            subfold = subFolders       
        if(i>0):
            for filename in files: 
                fin = open(os.path.join(root,filename), 'r')
                data=fin.read().replace('\n', '')
                data_frame.append(data)
                if subfold[i-1] not in target_hash:
                    target_hash[subfold[i-1]] = topic
                    topic += 1
                data_frame_target.append(target_hash[subfold[i-1]])
                data = ""
                fin.close()
        i=i+1
    return data_frame, np.array(data_frame_target,dtype=np.int32)
rootdir_train = "/run/media/avideep/Films/Important/MSc_Project/datasets/Reuters21578-Apte-90Cat/training/"
X_train,y_train = build_data_frame(rootdir_train)
rootdir_test = "/run/media/avideep/Films/Important/MSc_Project/datasets/Reuters21578-Apte-90Cat/test/"
X_test,y_test = build_data_frame(rootdir_test)
tfidf = TfidfVectorizer(min_df=3)
X_train = tfidf.fit_transform(X_train)
X_test_new = []
y_test_new = []
i=0
for idx, doc in enumerate(X_test):
    try:
        tfidf.transform(list(doc))
        X_test_new.append(doc)
        y_test_new.append(y_test[i])
    except UnicodeDecodeError:
        pass
    i = i + 1
y_test = y_test_new
X_test = tfidf.transform(X_test_new)
#X_train = features.fit_transform(X_train,y_train)
#X_test = features.transform(X_test)
print(X_train.shape)
print(X_test.shape)

In [ ]:
rootdir_train = "/run/media/avideep/Films/Important/MSc_Project/datasets/Reuters21578-Apte-90Cat/training/"
rootdir_test = "/run/media/avideep/Films/Important/MSc_Project/datasets/Reuters21578-Apte-90Cat/test/"
train_data = load_files(rootdir_train)
test_data = load_files(rootdir_test)

In [ ]:
X_train = train_data.data
y_train = train_data.target
X_test, y_test = test_data.data, test_data.target
tfidf = TfidfVectorizer(min_df=3)
X_train = tfidf.fit_transform(X_train)
X_test_new = []
y_test_new = []
for idx, doc in enumerate(X_test):
    try:
        tfidf.transform(list(doc))
        X_test_new.append(doc)
        y_test_new.append(y_test[idx])
    except UnicodeDecodeError:
        pass
y_test = []
y_test = deepcopy(y_test_new)
X_test = tfidf.transform(X_test_new)
#X_train = features.fit_transform(X_train,y_train)
#X_test = features.transform(X_test)
print(X_train.shape)
print(X_test.shape)

In [ ]:
from collections import Counter
c = Counter(y_test)
c

# 20 News Group

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroup_train = fetch_20newsgroups(subset='train')
X_train = newsgroup_train.data
y_train = newsgroup_train.target
newsgroup_test = fetch_20newsgroups(subset='test')
X_test = newsgroup_test.data
y_test = newsgroup_test.target
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df = 3)
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
features = SelectKBest(chi2, k = 1000)
X_train = features.fit_transform(X_train,y_train)
X_test = features.transform(X_test)
print(X_train.shape)
print(X_test.shape)

# tknn

In [27]:
def tknn(Xtrain,ytrain,xtest,beta,theta):
    L = beta
    S = []
    class_names = np.unique(ytrain)
    class_count = np.empty(class_names.shape[0])
    for xtrain in Xtrain:
        dist = distance.cosine(xtrain,xtest)
        if dist > theta:
            S.append(dist)
    SPN_index = np.argsort(S)
    SPN_labels = np.empty(SPN_index.shape[0]) 
    for i in range(SPN_index.shape[0]):
        SPN_labels[i] = ytrain[SPN_index[i]]
    SPN_0_labels = SPN_labels[:L]
    for i in range(class_names.shape[0]):
        val = 0
        for j in range(SPN_0_labels.shape[0]):
            if SPN_0_labels[j] == class_names[i]:
                val = val + 1
        class_count[i] = val
    while L <= SPN_labels.shape[0]:
        L_x1 = np.max(class_count)
        L_x1_class = np.argmax(class_count)
        L_x2 = np.max(np.delete(class_count,np.argmax(class_count)))
        L_x2_class = np.argmax(np.delete(class_count,np.argmax(class_count)))
        if L_x1 - L_x2 == beta:
            return L_x1_class, L
        else:
            L = L + 1
            c = SPN_labels[L]
            #print(type(c))
            class_count[int(c)] += 1
    return -1, L

In [57]:
X,y,number_of_class_labels = load_karypis('tr12',path="/run/media/avideep/Films/Important/MSc_Project/datasets/karypis",min_df=3)
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.20,random_state=1)
train_index, test_index = next(sss.split(X,y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
X_train, X_test = X_train.toarray(), X_test.toarray()

In [60]:
predicted_class_label = list(np.empty(len(X_test)))
L = list(np.empty(len(X_test)))
for i in range(len(X_test)):
    predicted_class_label[i], L[i] = tknn(X_train, y_train, X_test[i], 
                                    beta = 2,
                                    theta = 0.001)
    #print ('Classified test data #' + str(i) + ' as ' + str(predicted_class_label[i]) + ' with L = ' + str(L[i]))
print('Average value of L is ' + str(int(np.mean(L) + 0.5)))
fm=f1_score(y_test, predicted_class_label, average='micro') 
print ('\n Micro Averaged F1-Score :'+str(fm))
acc=accuracy_score(y_test, predicted_class_label) 
print ('\n Accuracy Score:'+str(acc))

Average value of L is 4

 Micro Averaged F1-Score :0.8095238095238095

 Accuracy Score:0.8095238095238095


# UCI Sentiment

In [160]:
amazon_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/sentiment labelled sentences/amazon_cells_labelled.txt'
imdb_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/sentiment labelled sentences/imdb_labelled.txt'
yelp_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/sentiment labelled sentences/yelp_labelled.txt'
sentiment_data = pd.read_csv(yelp_data_dir,sep='\t',header=None)
X,y = sentiment_data[0].values.tolist(), sentiment_data[1].values.tolist()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state =1)
#sss = StratifiedShuffleSplit(n_splits=1,test_size=0.20,random_state=1)
#train_index, test_index = next(sss.split(X,y))
#X_train, X_test = X[train_index], X[test_index]
#y_train, y_test = y[train_index], y[test_index]
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
print(X_train.shape)
print(X_test.shape)

(800, 1777)
(200, 1777)


# spambase

In [2]:
spambase_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/spambase/spambase.data'
spam_data = pd.read_csv(spambase_data_dir,sep=',',header=None)
#X,y = spam_data[0].values.tolist(), sentiment_data[1].values.tolist()
y = spam_data[57].values.tolist()
X = spam_data.drop([57],axis = 1).values.tolist()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state =1)

# Karypis

In [4]:
X,y,number_of_class_labels = load_karypis('tr45',path="/media/avideep/Films/Important/MSc_Project/datasets/karypis",min_df=3)
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.20,random_state=1)
train_index, test_index = next(sss.split(X,y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)
X_train, X_test = X_train.toarray(), X_test.toarray()

# IRIS

In [82]:
iris = load_iris()
X,y = iris.data, iris.target
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.20,random_state=1)
train_index, test_index = next(sss.split(X,y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

# Breast Cancer UCI

In [39]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X,y = data.data, data.target
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.20,random_state=1)
train_index, test_index = next(sss.split(X,y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
print(X.shape)
print(len(data.target_names))

(569, 30)
2


# Ecoli UCI

In [2]:
ecoli_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/ecoli.data'
data = pd.read_csv(ecoli_data_dir,sep=',',header=None)
X = np.array(data.drop([0,data.shape[1]-1],axis=1).values.tolist())
a = list(np.unique(data.iloc[:,-1]))
le = LabelEncoder()
le.fit(a)
y = le.transform(data.iloc[:,-1].values.tolist())
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.20,random_state=1)
train_index, test_index = next(sss.split(X,y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
print(X.shape)
print(len(a))

(336, 7)
8


In [110]:
len(X_test)

68

# ionosphere

In [161]:
ionosphere_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/ionosphere.data'
data = pd.read_csv(ionosphere_data_dir,sep=',',header=None)
X = np.array(data.drop([data.shape[1]-1],axis=1).values.tolist())
a = list(np.unique(data.iloc[:,-1]))
le = LabelEncoder()
le.fit(a)
y = le.transform(data.iloc[:,-1].values.tolist())
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.20,random_state=1)
train_index, test_index = next(sss.split(X,y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
print(X.shape)
print(len(a))

(351, 34)
2


# abalone

In [3]:
abalone_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/abalone.txt'
data = pd.read_csv(abalone_data_dir,sep=',',header=None)
X = np.array(data.drop([0,data.shape[1]-1],axis=1).values.tolist())
a = list(np.unique(data.iloc[:,-1]))
le = LabelEncoder()
le.fit(a)
y = le.transform(data.iloc[:,-1].values.tolist())
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state =1)
print(X.shape)
print(len(a))

(4177, 7)
28


# Pima India Diabetes

In [67]:
pid_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/PimaIndiaDiabetes.txt'
data = pd.read_csv(pid_data_dir,sep=',',header=None)
X = np.array(data.drop([data.shape[1]-1],axis=1).values.tolist())
a = list(np.unique(data.iloc[:,-1]))
le = LabelEncoder()
le.fit(a)
y = le.transform(data.iloc[:,-1].values.tolist())
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state =1)
print(X.shape)
print(len(a))

(768, 8)
2


# Vowel Context

In [19]:
vowel_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/vowel-context.data'
data = pd.read_csv(vowel_data_dir,sep=',',header=None)
data_train = data[data[0] == 0]
data_test = data[data[0] == 1]
X_train = np.array(data_train.drop([0,data.shape[1]-1],axis=1).values.tolist())
X_test = np.array(data_test.drop([0,data.shape[1]-1],axis=1).values.tolist())
y_train = data_train.iloc[:,-1]
y_test = data_test.iloc[:,-1]

# glass UCI

In [222]:
glass_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/glass.data'
data = pd.read_csv(glass_data_dir,sep=',',header=None)
X = np.array(data.drop([0,data.shape[1]-1],axis=1).values.tolist())
y =np.array(data.iloc[:,-1].values.tolist())
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.20,random_state=1)
train_index, test_index = next(sss.split(X,y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
print(X.shape)

(214, 9)


# Lung Cancer UCI

In [233]:
lung_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/lung-cancer.data'
data = pd.read_csv(lung_data_dir,sep=',',header=None)
data_dropped = data.drop([4,38],axis = 1)
X = np.array(data_dropped.drop([0],axis=1).values.tolist())
y = np.array(data_dropped[0].values.tolist())
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.20,random_state=1)
train_index, test_index = next(sss.split(X,y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
print(X.shape)
print(len(np.unique(y)))

(32, 54)
3


# Heart Disease

In [401]:
lung_data_dir = '/run/media/avideep/Films/Important/MSc_Project/datasets/processed.cleveland.data'
data = pd.read_csv(lung_data_dir,sep=',',header=None)
X = np.array(data.drop([data.shape[1]-1],axis=1).values.tolist())
y =np.array(data.iloc[:,-1].values.tolist())
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.20,random_state=1)
train_index, test_index = next(sss.split(X,y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

# Image Segmentation UCI

# Algorithm

In [11]:
class_names = list(np.unique(y_train))
class_names_dict = {}
for i,val in enumerate(class_names):
    class_names_dict[i] = val

In [12]:
def findMedians(X_train,y_train):
    medians = {}
    for class_name in class_names:
        class_name_x_train = [X_train[i] for i,val in enumerate(y_train) if val == class_name]
        mean = []
        for i in range(len(class_name_x_train[0])):
            col = []
            for row in class_name_x_train:
                col.append(row[i])
            mean.append(np.mean(np.array(col).astype(np.float)))
        median_row = deepcopy(class_name_x_train[0])
        for row in class_name_x_train:
            if distance.cosine(np.array(row).astype(np.float),np.array(mean).astype(np.float)) < distance.cosine(np.array(median_row).astype(np.float),np.array(mean).astype(np.float)):
                median_row = deepcopy(row)
        medians[class_name] = np.array(median_row).astype(np.float)
    return medians

def findSelectedSamples(X_train,y_train,x_test,medians,method2):
    X_train = np.array(X_train).astype(np.float)
    x_test = np.array(x_test).astype(np.float)
    x_train_selected = []
    y_train_selected = []
    if method2 == 1: 
        median_list = list(np.empty(len(class_names)))
        for i in range(len(class_names)):
            median_list[i] = distance.cosine(medians[class_names_dict[i]],x_test)
        min_median = max(median_list)
        for i in range(len(X_train)):
            if(distance.cosine(X_train[i],x_test)<=distance.cosine(min_median,x_test)):
            #if(distance.cosine(X_train[i],x_test)<=distance.cosine(medians[y_train[i]],x_test)):
                x_train_selected.append(X_train[i])
                y_train_selected.append(y_train[i])
    elif method2 == 0:
        for i in range(len(X_train)):
            #if(distance.cosine(X_train[i],x_test)<=distance.cosine(min_median,x_test)):
            if(distance.cosine(X_train[i],x_test)<=distance.cosine(medians[y_train[i]],x_test)):
                x_train_selected.append(X_train[i])
                y_train_selected.append(y_train[i])
    return np.array(x_train_selected), y_train_selected

def sortneighbors(x,y,x_test):
    x = np.array(x).astype(np.float)
    x_test = np.array(x_test).astype(np.float)
    dist = np.empty(len(x))
    for i in range(len(x)):
        dist[i] = distance.cosine(x[i],x_test)
    dist = np.argsort(dist)
    x_sorted = np.empty(shape = (len(x),len(X_train[1])))
    y_sorted = []
    k = 0
    for i in dist:
        x_sorted[k] = x[i]
        y_sorted.append(y[i])
        k = k + 1
    return x_sorted,y_sorted

def cosine_weighted_tkNN(X_train,y_train,x_test,medians,gamma=0.025,l=2,method=1,method2=1):
    y_test = -1
    x_train_selected, y_train_selected = findSelectedSamples(X_train,y_train,x_test,medians,method2) 
    x_train_sorted, y_train_sorted = sortneighbors(x_train_selected,y_train_selected,x_test)
    x_test = np.array(x_test).astype(np.float)  
    x_train_sorted = np.array(x_train_sorted).astype(np.float)
    L = l
    scaler = MinMaxScaler()
    while(L <= len(x_train_sorted)):
        S_L = x_train_sorted[:L]
        S_L_y = y_train_sorted[:L]
        weights = np.empty(len(S_L))
        weights_temp = []
        if method == 1:
            for i in range(len(S_L)):
                weights_temp.append([distance.cosine(S_L[i],x_test), distance.cosine(medians[S_L_y[i]],S_L[i])])
            weights_transformed = scaler.fit_transform(weights_temp)
            for i in range(len(S_L)):
                #weights[i] = ((1 - distance.cosine(S_L[i],x_test))*(1 - distance.cosine(medians[S_L_y[i]],x_test)))
                weights[i] = (1 - weights_transformed[i][0]) * (1 - weights_transformed[i][1])
        elif method == 0:
            for i in range(len(S_L)):
                weights[i] = ((1 - distance.cosine(S_L[i],x_test))*(1 - distance.cosine(medians[S_L_y[i]],x_test)))
        class_weights = np.zeros(len(class_names))
        class_cardinality = np.zeros(len(class_names))
        p = 0
        for class_name in class_names:
            class_weights[p] = np.sum([weights[k] for k in np.where(S_L_y == class_name)])
            class_cardinality[p] = np.sum([1 for k in np.where(S_L_y == class_name)])                                                   
            p = p + 1 
        max1 = max(class_weights)
        max1_index = np.argmax(class_weights)
        if class_cardinality[max1_index] == 0:
            nmax1 = 0
        else:
            nmax1 = max1 / class_cardinality[max1_index]
        class_weights_max_containing = list(class_weights)
        class_weights_temp = list(class_weights)
        class_weights_temp.remove(max1)
        max2 = max(class_weights_temp)
        max2_index = np.argmax(class_weights_temp)
        if class_cardinality[max2_index] == 0:
            nmax2 = 0
        else:
            nmax2 = max2 / class_cardinality[max2_index]
        if(nmax1 - nmax2 >=gamma):
            y_test = class_names[class_weights_max_containing.index(max1)]
            break
        else:
            L = L + 1
    return y_test, L

In [23]:
X_train, X_test = X_train.toarray(), X_test.toarray()

In [9]:
medians = findMedians(X_train,y_train)

NameError: name 'findMedians' is not defined

In [28]:
predicted_class_label = list(np.empty(len(X_test)))
L = list(np.empty(len(X_test)))
for i in range(len(X_test)):
    predicted_class_label[i] = tknn(X_train, y_train, X_test[i], 
                                    beta = 3,
                                    theta = 0.001)
    print ('Classified test data #' + str(i) + ' as ' + str(predicted_class_label[i]) + ' with L = ' + str(L[i]))
print('Average value of L is ' + str(int(np.mean(L) + 0.5)))

Classified test data #0 as (4, 3) with L = 6.91075581265556e-310
Classified test data #1 as (7, 9) with L = 6.91075581265556e-310
Classified test data #2 as (4, 3) with L = 4.6439151246985e-310
Classified test data #3 as (2, 3) with L = 4.6439151246985e-310
Classified test data #4 as (1, 3) with L = 0.0
Classified test data #5 as (3, 3) with L = 0.0
Classified test data #6 as (8, 3) with L = nan
Classified test data #7 as (6, 3) with L = 0.0
Classified test data #8 as (0, 3) with L = 0.0
Classified test data #9 as (1, 3) with L = 5.4e-323
Classified test data #10 as (4, 3) with L = 6.91068361762017e-310
Classified test data #11 as (8, 3) with L = 4.6439133248577e-310
Classified test data #12 as (4, 3) with L = 0.0
Classified test data #13 as (3, 3) with L = 0.0
Classified test data #14 as (0, 3) with L = nan
Classified test data #15 as (3, 3) with L = 0.0
Classified test data #16 as (5, 3) with L = 3.16e-322
Classified test data #17 as (0, 3) with L = 5.4e-323
Classified test data #18 

ValueError: cannot convert float NaN to integer

In [26]:
#predicted_class_label_temp = [predicted_class_label[i] for i,val in enumerate(predicted_class_label) if val != -1]
#predicted_class_label = deepcopy(predicted_class_label_temp)
#y_test_temp = [y_test[i] for i,val in enumerate(predicted_class_label) if val != -1]
#y_test = deepcopy(y_test_temp)
print ('Classification of the test samples')
# print ('Evaluation using Precision, Recall and F-measure (micro)')  
# pr=precision_score(y_test, predicted_class_label, average='micro')
# print ('\n Precision:'+str(pr))
# re=recall_score(y_test, predicted_class_label, average='micro')
# print ('\n Recall:'+str(re))
# fm=f1_score(y_test, predicted_class_label, average='micro') 
# print ('\n F-measure:'+str(fm))

# print ('Evaluation using Precision, Recall and F-measure (macro)')
# pr=precision_score(y_test, predicted_class_label, average='macro')
# print ('\n Precision:'+str(pr))
# re=recall_score(y_test, predicted_class_label, average='macro')
# print ('\n Recall:'+str(re))
fm=f1_score(y_test, predicted_class_label, average='macro') 
print ('\n Macro Averaged F1-Score :'+str(fm))
acc=accuracy_score(y_test, predicted_class_label) 
print ('\n Accuracy Score:'+str(acc))

Classification of the test samples

 Macro Averaged F1-Score :0.8377046400797787

 Accuracy Score:0.9202898550724637
